#  Tensorflow training 
In this tutorial, you will train a mnist model in TensorFlow.

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)
* Go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)
[//]: # * Review the [tutorial](../train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb) on single-node TensorFlow training using the SDK
* install CMAKS SDK

In [ ]:
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-contrib-k8s-preview/D58E86006C65 azureml-contrib-k8s


In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.18.0


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [18]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [19]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: akse-attach-ws1
Azure region: eastus2euap
Subscription id: 5abfd9c4-ec8c-4db9-acd4-c762dce93508
Resource group: akse-ws-rg


## Specify existing Kubernetes Compute

In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

# choose a name for your Kubernetes compute
compute_name = 'aks-e-profile1'
compute_target = ComputeTarget(workspace=ws, name=compute_name)

In [21]:
compute_target

ArcKubernetesCompute(workspace=Workspace.create(name='akse-attach-ws1', subscription_id='5abfd9c4-ec8c-4db9-acd4-c762dce93508', resource_group='akse-ws-rg'), name=aks-e-profile1, id=/subscriptions/5abfd9c4-ec8c-4db9-acd4-c762dce93508/resourceGroups/akse-ws-rg/providers/Microsoft.MachineLearningServices/workspaces/akse-attach-ws1/computes/aks-e-profile1, type=ArcKubernetes, provisioning_state=Succeeded, location=eastus2euap, tags=None)

## Create a Dataset for Files
A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. Dataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [6]:
#initialize file dataset 
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

you may want to register datasets using the register() method to your workspace so they can be shared with others, reused across various experiments, and referred to by name in your training script.

In [7]:
#register dataset to workspace
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset for Arc',
                           description='training and test dataset',
                           create_new_version=True)

In [8]:
# list the files referenced by dataset
dataset.to_path()

['/http%3A/%2Fyann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

## Train model on the Kubernetes compute

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [9]:
import os
script_folder = './tf-resume-training'
os.makedirs(script_folder, exist_ok=True)

Copy the training script `tf_mnist_with_checkpoint.py` into this project directory.

In [14]:
import shutil

# the training logic is in the tf_mnist_with_checkpoint.py file.
shutil.copy('/home/azureuser/cloudfiles/code/Users/saudas/tf_mnist_with_checkpoint.py', script_folder)
#/mnt/batch/tasks/shared/LS_root/mounts/clusters/exp109/code/users/saudas/Distributed-training/tf-resume-training
# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('/home/azureuser/cloudfiles/code/Users/saudas/utils.py', script_folder)

'./tf-resume-training/utils.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [22]:
from azureml.core import Experiment

experiment_name = 'akse-arc-tf-training1'
experiment = Experiment(ws, name=experiment_name)

### Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

In [23]:
from azureml.train.dnn import TensorFlow

script_params={
    '--data-folder': dataset.as_named_input('mnist').as_mount()
}

estimator= TensorFlow(source_directory=script_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_mnist_with_checkpoint.py',
                      use_gpu=False,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.


In the above code, we passed our training data reference `ds_data` to our script's `--data-folder` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### Submit job
### Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [24]:
run = experiment.submit(estimator)
print(run)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


Run(Experiment: akse-arc-tf-training1,
Id: akse-arc-tf-training1_1605469053_3103f536,
Type: azureml.scriptrun,
Status: Starting)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.